In [103]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import cleaning_dicts
import os

In [104]:
print(cleaning_dicts.clean_team_pff)

{'arz': 'ari', 'blt': 'bal', 'clv': 'cle', 'hst': 'hou', 'la': 'lar', 'sd': 'lac', 'sl': 'lar'}


In [105]:
cur_dir = os.path.abspath(os.getcwd())
cur_week_folder = 'nfl_week_11'
cur_week = '11'
int_cur_week = 11

In [106]:
print(cur_dir)

/media/tom/Windows/Users/booth/Documents/spreads/spreads_2021


In [107]:
spreads = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/spreadspoke_w'+cur_week+'.csv')
weather = pd.read_csv(cur_dir+"/weather/"+'/weather_hist_w'+cur_week+'.csv')
fo = pd.read_csv(cur_dir+"/other_data/"+'/fo_final.csv')

In [108]:
def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)


### Read in raw datasets

In [109]:
qb_standard_2019 = pd.read_csv(cur_dir+'/historic_data'+'/qb_standard_hist.csv')
oline_2019 = pd.read_csv(cur_dir+'/historic_data'+'/ol_standard_hist.csv')
#qb_adj_2019 = pd.read_csv(cur_dir+'historic_data'+'/qb_adjusted_hist_16.csv')
rush_std_2019 = pd.read_csv(cur_dir+'/historic_data'+'/rush_standard_hist.csv')
rec_std_2019 = pd.read_csv(cur_dir+'/historic_data'+'/rec_standard_hist.csv')
def_standard_2019 = pd.read_csv(cur_dir+'/historic_data'+'/def_standard_hist.csv')

qb_standard = qb_standard_2019[~qb_standard_2019['player'].isnull()]
oline = oline_2019[~oline_2019['player'].isnull()]
#qb_adj_2019 = qb_adj_2019[~qb_adj_2019['player'].isnull()]
rush_std = rush_std_2019[~rush_std_2019['player'].isnull()]
rec_std = rec_std_2019[~rec_std_2019['player'].isnull()]
def_standard = def_standard_2019[~def_standard_2019['player'].isnull()]


qb_standard_2021 = pd.read_csv(cur_dir+'/historic_data'+'/qb_standard_hist_w'+cur_week+'.csv')
oline_2021 = pd.read_csv(cur_dir+'/historic_data'+'/ol_standard_hist_w'+cur_week+'.csv')
#qb_adj_2019 = pd.read_csv(cur_dir+'historic_data'+'/qb_adjusted_hist_16.csv')
rush_std_2021 = pd.read_csv(cur_dir+'/historic_data'+'/rush_standard_hist_w'+cur_week+'.csv')
rec_std_2021 = pd.read_csv(cur_dir+'/historic_data'+'/rec_standard_hist_w'+cur_week+'.csv')
def_standard_2021 = pd.read_csv(cur_dir+'/historic_data'+'/def_standard_hist_w'+cur_week+'.csv')

qb_standard_2021 = qb_standard_2021[~qb_standard_2021['player'].isnull()]
oline_2021 = oline_2021[~oline_2021['player'].isnull()]
#qb_adj_2019 = qb_adj_2019[~qb_adj_2019['player'].isnull()]
rush_std_2021 = rush_std_2021[~rush_std_2021['player'].isnull()]
rec_std_2021 = rec_std_2021[~rec_std_2021['player'].isnull()]
def_standard_2021 = def_standard_2021[~def_standard_2021['player'].isnull()]

qb_standard = pd.concat([qb_standard, qb_standard_2021], axis=0)
oline = pd.concat([oline, oline_2021], axis=0)
#qb_adj = pd.concat([qb_adj, qb_adj_old], axis=0)
rush_std = pd.concat([rush_std, rush_std_2021], axis=0)
rec_std = pd.concat([rec_std, rec_std_2021], axis=0)
def_standard = pd.concat([def_standard, def_standard_2021], axis=0)

qb_standard.reset_index(inplace=True, drop=True)
oline.reset_index(inplace=True, drop=True)
rush_std.reset_index(inplace=True, drop=True)
rec_std.reset_index(inplace=True, drop=True)
def_standard.reset_index(inplace=True, drop=True)



qb_standard=qb_standard.fillna(0.1)
oline=oline.fillna(0.1)
rush_std=rush_std.fillna(0.1)
rec_std=rec_std.fillna(0.1)
def_standard=def_standard.fillna(0.1)

In [110]:
qb_standard.drop('player_id', axis=1, inplace=True)
oline.drop('player_id', axis=1, inplace=True)
rush_std.drop('player_id', axis=1, inplace=True)
rec_std.drop('player_id', axis=1, inplace=True)
def_standard.drop('player_id', axis=1, inplace=True)

### Bring them the data together and remove duplicate captures

In [111]:


qb_standard.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)
oline.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)
#qb_adj.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)
rush_std.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)
rec_std.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)
def_standard.drop_duplicates(subset=['player','year','week'], keep='first', inplace=True)

def clean_str_nan(nData):
    nData.replace(["NaN", 'NaT'], np.nan, inplace = True)
    nData=nData[pd.notnull(nData['player'])]
    #nData=nData.dropna(thresh=10)
    nData['year'] = nData['year'].astype(int)
    nData['week'] = nData['week'].astype(int)
    return nData

rush_std = clean_str_nan(rush_std)
qb_standard =clean_str_nan(qb_standard)
oline=clean_str_nan(oline)
#qb_adj=clean_str_nan(qb_adj)
rec_std=clean_str_nan(rec_std)
def_standard=clean_str_nan(def_standard)


In [112]:
def clean_name(df):
    result = ''.join(c for c in df['player'] if c.isalpha())
    return result.lower()

In [113]:
fo.columns

Index(['team_id', 'year', 'week', 'team', 'w-l', 'off_dvoa_rank', 'off_dvoa',
       'off_weight_dvoa_rank', 'off_weight_dvoa', 'pass_off_rank', 'pass_off',
       'rush_off_rank', 'rush_off', 'def_dvoa_rank', 'def_dvoa',
       'def_weight_dvoa_rank', 'def_weight_dvoa', 'pass_def_rank', 'pass_def',
       'rush_def_rank', 'rush_def'],
      dtype='object')

In [114]:
def raiders_fo(df):
    if 'oak' in str(df.team) and '2020' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    elif 'oak' in str(df.team) and '2021' in str(df.year):
        return df.team_id.str.replace("oak_","lv_")
    else:
        return str(df.team_id)


In [115]:
fo_2021 = pd.read_csv(cur_dir+'/football_outsiders/fo_final.csv')
fo_pp = pd.read_csv(cur_dir+'/current_season_data/'+cur_week_folder+'/fo_clean_pp_w'+cur_week+'.csv')
fo = pd.concat([fo, fo_2021, fo_pp], axis=0)

fo['team_id'] = fo.apply(lambda df: raiders_fo(df), axis=1)
#fo['team_id']=fo['team_id'].str.replace("oak_","lv_")

fo = fo.sort_values(by=["team","year","week"], ascending=[True, True, True])
#data=data.fillna(data.mean())
def running_diff(arr, N):
    return np.array([arr[i] - arr[i-N] for i in range(N, len(arr))])

fo['off_dvoa_momentum'] = np.hstack((fo['off_dvoa_rank'].values[:3], running_diff(fo['off_dvoa_rank'].values, 3)))
fo['off_wt_dvoa_momentum'] = np.hstack((fo['off_weight_dvoa_rank'].values[:3], running_diff(fo['off_weight_dvoa_rank'].values, 3)))
fo['pass_off_dvoa_momentum'] = np.hstack((fo['pass_off_rank'].values[:3], running_diff(fo['pass_off_rank'].values, 3)))
fo['rush_off_dvoa_momentum'] = np.hstack((fo['rush_off_rank'].values[:3], running_diff(fo['rush_off_rank'].values, 3)))

fo['def_dvoa_momentum'] = np.hstack((fo['def_dvoa_rank'].values[:3], running_diff(fo['def_dvoa_rank'].values, 3)))
fo['def_wt_dvoa_momentum'] = np.hstack((fo['def_weight_dvoa_rank'].values[:3], running_diff(fo['def_weight_dvoa_rank'].values, 3)))
fo['pass_def_dvoa_momentum'] = np.hstack((fo['pass_def_rank'].values[:3], running_diff(fo['pass_def_rank'].values, 3)))
fo['rush_dvoa_momentum'] = np.hstack((fo['rush_def_rank'].values[:3], running_diff(fo['rush_def_rank'].values, 3)))

num_cols = fo.select_dtypes(include=[np.number]).columns.tolist()
ids = pd.DataFrame(fo.select_dtypes(exclude=[np.number])).reset_index(drop=True)
roll3 = fo.groupby(['team','year'])[num_cols].apply(lambda x : x.shift(+1)).reset_index(drop=True)
fo_roll = pd.concat([ids, roll3], axis=1)

In [116]:
fo_roll.tail().T

,8507,8508,8509,8510,8511
team_id,was_2021_7,was_2021_8,was_2021_9,was_2021_10,was_2021_11
team,was,was,was,was,was
w-l,2-5,2-6,2-6,3-6,3-6
year,2021,2021,2021,2021,2021
week,6,7,8,9,10
off_dvoa_rank,21,23,23,22,20
off_dvoa,-6.1,-6.7,-6.5,-4.8,-1.8
off_weight_dvoa_rank,21,24,23,22,20
off_weight_dvoa,-5.9,-7.1,-6.6,-6,-2.8
pass_off_rank,20,22,23,23,19


def rolling_fos(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
        
fo_roll = rolling_fos(data=fo, roll_value=3, roll_type='mean')
fo_roll.head().T

### Clean Spread Data

In [100]:
spreads['schedule_season']=spreads['schedule_season'].apply(int)
spreads['schedule_week']=spreads['schedule_week'].apply(int)
spreads.head().T

,0,1,2,3,4
schedule_date,9/10/2006,9/10/2006,9/10/2006,9/10/2006,9/10/2006
schedule_season,2006,2006,2006,2006,2006
schedule_week,1,1,1,1,1
schedule_playoff,False,False,False,False,False
team_home,Kansas City Chiefs,New York Giants,St. Louis Rams,Arizona Cardinals,Detroit Lions
team_home_abb,KC,NYG,LAR,ARI,DET
score_home,10,21,18,34,6
score_away,23,26,10,27,9
team_away,Cincinnati Bengals,Indianapolis Colts,Denver Broncos,San Francisco 49ers,Seattle Seahawks
away_team_abb,CIN,IND,DEN,SF,SEA


In [101]:
spreads = spreads[spreads['schedule_season']>=2006]
spreads = spreads[['schedule_date','schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)

def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)

##  functions to clean and scrub data by source  
def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)        
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
spreads = clean_spreads(spreads)
spreads = pd.merge(spreads, weather, on='away_matchup_id', how='left')
spreads['fav_cover'].value_counts()

sh = spreads
sa = spreads

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)

## add nfl team data  ##
nfl_team = pd.read_csv(cur_dir+'/other_data/nfl_teams.txt', sep='\t')
nfl_team['team_ids'] = nfl_team['team_ids'].str.lower()
nfl_team = nfl_team[['team_ids','team_conference','team_division']]
conf_dict = dict(zip(nfl_team['team_ids'], nfl_team['team_conference']))
div_dict = dict(zip(nfl_team['team_ids'], nfl_team['team_division']))


spread_comb = pd.merge(spread_comb, nfl_team, left_on='team_abb', right_on='team_ids', how='left')
spread_comb = spread_comb[spread_comb['team_favorite_id'] != 'pick']
spread_comb = pd.merge(spread_comb, fo_roll, on='team_id', how='left')

def div_conf_game(nData, dict_type):
    abb1 = nData['team_favorite_id']
    abb2 = nData['team_notfav_id']
    for k,v in dict_type.items():
        if abb1 == k:
            v_fav = v
        elif abb2 == k:
            v_dog = v
        else:
            pass
    #print(abb1, abb2, v_fav, v_dog)
    if v_fav == v_dog:
        return 1
    else:
        return 0
#spread_comb['fav_conf_game'] = spread_comb.apply(lambda nData: div_conf_game(nData, conf_dict), axis=1)
#spread_comb['fav_div_game'] = spread_comb.apply(lambda nData: div_conf_game(nData, div_dict), axis=1)

In [102]:
spread_for_pff = spread_comb[['home_matchup_id',
                              'schedule_season',
                              'schedule_week',
                              'team_home_abb',
                              'score_home',
                              'score_away',
                              'away_team_abb',
                              'team_favorite_id',
                              'spread_favorite',
                              'over_under_line',
                              'team_notfav_id',
                              'fav_cover',
                              'over_under_result',
                              'fav_homeoraway']]

spread_comb.head(n=5).T

,8461,8462,8463,8464,8465
home_matchup_id,lvvscin_2021_11,kcvsdal_2021_11,seavsari_2021_11,lacvspit_2021_11,tbvsnyg_2021_11
away_matchup_id,cin@lv_2021_11,dal@kc_2021_11,ari@sea_2021_11,pit@lac_2021_11,nyg@tb_2021_11
team_id,cin_2021_11,dal_2021_11,ari_2021_11,pit_2021_11,nyg_2021_11
schedule_date,nan,nan,nan,nan,nan
schedule_season,2021,2021,2021,2021,2021
schedule_week,11,11,11,11,11
team_home_abb,lv,kc,sea,lac,tb
score_home,nan,nan,nan,nan,nan
score_away,nan,nan,nan,nan,nan
away_team_abb,cin,dal,ari,pit,nyg


In [18]:
#tgs_historic = pd.read_csv(cur_dir+'/historic_data/team_game_summaries_hist.csv')
tgs_historic = pd.read_csv(cur_dir+'/historic_data/team_game_summaries_historic.csv')
#tgs_historic = pd.concat([tgs_historic, tgs_historic_old], axis=0)
tgs_2019 = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/team_game_summaries_2021_w'+cur_week+'.csv')
tgs_2019 = tgs_2019[(tgs_2019['year'] == 2021) & (tgs_2019['week'] <= int(cur_week))]

In [19]:
tgs_historic.tail()

,team,year,week,home_or_away,team_num,game_id,opponent,date,time,wl,...,pass_blocking,receiving,rushing,run_blocking,defense,rush_defense,tackling,pass_rush,coverage,special_teams
7659,washington-football-team,2020,13,@,32,18716,Pittsburgh Steelers,12/07,5:00pm,W,...,53.8,62.0,42.7,57.7,75.5,81.0,55.5,68.8,69.9,50.5
7660,washington-football-team,2020,14,@,32,18737,San Francisco 49ers,12/13,4:25pm,W,...,86.0,57.7,68.4,79.1,63.5,49.8,76.7,74.4,63.5,66
7661,washington-football-team,2020,15,NaN,32,18751,Seattle Seahawks,12/20,1:00pm,L,...,83.7,61.1,68.1,44.1,68.2,56.0,58.3,66.7,73.5,78
7662,washington-football-team,2020,16,NaN,32,18767,Carolina Panthers,12/27,4:05pm,L,...,84.1,56.3,74.8,70.8,76.8,75.5,47.1,82.7,66.6,55.1
7663,washington-football-team,2020,17,@,32,18783,Philadelphia Eagles,01/03,8:20pm,W,...,75.8,57.8,65.8,64.3,64.9,70.3,69.7,62.3,59.0,72.7


In [20]:
tgs = pd.concat([tgs_historic, tgs_2019], axis=0).reset_index(drop=True)

In [21]:
tgs.tail()

,team,year,week,home_or_away,team_num,game_id,opponent,date,time,wl,...,pass_blocking,receiving,rushing,run_blocking,defense,rush_defense,tackling,pass_rush,coverage,special_teams
7959,washington-football-team,2021,5,NaN,32,19679,New Orleans Saints,10/10,1:00pm,L,...,81.2,69.8,76.9,65.7,67.3,75.4,67.1,77.6,48.8,72.3
7960,washington-football-team,2021,6,NaN,32,19695,Kansas City Chiefs,10/17,1:00pm,L,...,83.0,56.0,55.0,73.6,52.1,51.3,55.0,67.1,48.4,47
7961,washington-football-team,2021,7,@,32,19704,Green Bay Packers,10/24,1:00pm,L,...,55.7,62.9,48.7,72.6,69.6,80.4,82.2,66.8,59.0,63.2
7962,washington-football-team,2021,8,@,32,19726,Denver Broncos,10/31,4:25pm,L,...,73.4,62.9,62.5,80.1,60.5,61.7,49.4,62.6,59.1,67.2
7963,washington-football-team,2021,10,NaN,32,19745,Tampa Bay Buccaneers,11/14,1:00pm,W,...,86.2,68.9,58.1,72.7,67.9,70.9,69.1,56.7,70.2,67.8


In [22]:
tgs['team_name'] = tgs['team'].map(cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

In [23]:
tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

#tgs = pd.concat([tgs_historic, tgs_2019], axis=0).reset_index(drop=True)
#tgs = pd.read_csv('U:/Data/firefox/gov/team_game_summaries_2020.csv')
##  functions to clean and scrub data by source  ##
def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

   
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    #df['home_or_away']=df['home_or_away'].str.replace('@','1')
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    #df.insert(4, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
   

    df = df.apply(pd.to_numeric, errors='ignore')
   
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_id_impute'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
   
    return df
   
tgs_clean = clean_pff_team_summ(tgs)

tgs_clean = pd.merge(tgs_clean, spread_for_pff, on='home_matchup_id', how='left')
tgs_clean.drop_duplicates(inplace=True)
tgs_clean.head().T


def hora1(nData):
    if nData['team_favorite_id'] == nData['team_name']:
        return 1
    else:
        return 0
tgs_clean['team_favored_or_not'] = tgs_clean.apply(lambda nData: hora1(nData), axis=1)

def team_cov(nData):
    if (nData['team_favorite_id'] == nData['team_name']) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['team_favorite_id'] != nData['team_name']) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0
tgs_clean['ats_w'] = tgs_clean.apply(lambda nData: team_cov(nData), axis=1)

def team_no_cov(nData):
    if (nData['team_favorite_id'] == nData['team_name']) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['team_favorite_id'] != nData['team_name']) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0
tgs_clean['ats_l'] = tgs_clean.apply(lambda nData: team_no_cov(nData), axis=1)

In [24]:
tgs_clean.head(n=10).T

,0,2,4,6,8,10,12,14,16,18
unique_team_id,ari_2006_1,ari_2006_2,ari_2006_3,ari_2006_4,ari_2006_5,ari_2006_6,ari_2006_7,ari_2006_8,ari_2006_10,ari_2006_11
team_id_impute,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006,ari_2006
opponent_id,sf_2006_1,sea_2006_2,lar_2006_3,atl_2006_4,kc_2006_5,chi_2006_6,oak_2006_7,gb_2006_8,dal_2006_10,det_2006_11
home_matchup_id,arivssf_2006_1,seavsari_2006_2,arivslar_2006_3,atlvsari_2006_4,arivskc_2006_5,arivschi_2006_6,oakvsari_2006_7,gbvsari_2006_8,arivsdal_2006_10,arivsdet_2006_11
team,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals,arizonacardinals
year,2006,2006,2006,2006,2006,2006,2006,2006,2006,2006
week,1,2,3,4,5,6,7,8,10,11
home_or_away,0,1,0,1,0,0,1,1,0,0
team_num,1,1,1,1,1,1,1,1,1,1
game_id,3752,3766,3784,3790,3812,3832,3841,3847,3886,3892


In [25]:
tgs_cur = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/tgs_clean_pp_w'+cur_week+'.csv')

tgs_clean = pd.concat([tgs_clean, tgs_cur], axis=0)

tgs_clean = tgs_clean[['unique_team_id','team_id_impute','wl','pf','pa',
                       'opponent_id','team_name','year','week','overall_performance', 'offense', 'pass',
       'pass_blocking', 'receiving', 'rushing', 'run_blocking', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage', 'special_teams',
                      'wl_int','fav_homeoraway','team_favored_or_not','ats_w','ats_l',]]

OU_ids = tgs_clean[['unique_team_id','wl','pf','opponent_id']]

tgs_clean = tgs_clean.drop(['wl'], axis=1)
tgs_clean_5 = tgs_clean.add_suffix('_overall_pff_5s')

tgs_clean= tgs_clean.add_suffix('_overall_pff')

tgs_clean.tail(n=10).T

,22,23,24,25,26,27,28,29,30,31
unique_team_id_overall_pff,no_2021_11,nyg_2021_11,nyj_2021_11,phi_2021_11,pit_2021_11,sea_2021_11,sf_2021_11,tb_2021_11,ten_2021_11,was_2021_11
team_id_impute_overall_pff,no_2021,nyg_2021,nyj_2021,phi_2021,pit_2021,sea_2021,sf_2021,tb_2021,ten_2021,was_2021
pf_overall_pff,21,23,17,30,16,0,31,19,23,29
pa_overall_pff,23,16,45,13,16,17,10,29,21,19
opponent_id_overall_pff,ten_2021_11,lv_2021_11,buf_2021_11,den_2021_11,det_2021_11,gb_2021_11,lar_2021_11,was_2021_11,no_2021_11,tb_2021_11
team_name_overall_pff,no,nyg,nyj,phi,pit,sea,sf,tb,ten,was
year_overall_pff,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021
week_overall_pff,11,11,11,11,11,11,11,11,11,11
overall_performance_overall_pff,73.9,65.5,53.7,75.8,60.3,58.9,70.2,61.4,68.2,76
offense_overall_pff,66.7,65.6,61.2,78.8,55.9,49.8,79.3,64.8,56.9,74.9


In [26]:
fo.head().T

,3,56,76,110,142
team_id,ari_2006_1,ari_2006_2,ari_2006_3,ari_2006_4,ari_2006_5
year,2006,2006,2006,2006,2006
week,1,2,3,4,5
team,ari,ari,ari,ari,ari
w-l,1-0,1-1,1-2,1-3,1-4
off_dvoa_rank,8,17,26,30,30
off_dvoa,19.3,-6.7,-13.6,-28.7,-23.6
off_weight_dvoa_rank,8,17,26,30,30
off_weight_dvoa,19.3,-6.7,-13.6,-28.7,-23.6
pass_off_rank,7,14,23,28,27


In [27]:
def rolling_tgs(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team_name_overall_pff","year_overall_pff","week_overall_pff"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(data['team_id_impute_overall_pff'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['team_id_impute_overall_pff'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['team_id_impute_overall_pff'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
        
tgs_roll = rolling_tgs(data=tgs_clean, roll_value=3, roll_type='mean')
tgs_roll= tgs_roll[~tgs_roll['year_overall_pff'].isin(['nan'])]

In [28]:
def rolling_tgs_5(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team_name_overall_pff_5s","year_overall_pff_5s","week_overall_pff_5s"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        roll5 = data.groupby(data['team_id_impute_overall_pff_5s'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll4 = data.groupby(data['team_id_impute_overall_pff_5s'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll3 = data.groupby(data['team_id_impute_overall_pff_5s'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll2 = data.groupby(data['team_id_impute_overall_pff_5s'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-3).mean())
        roll1 = data.groupby(data['team_id_impute_overall_pff_5s'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-4).mean())
        roll5 = pd.DataFrame(roll5.combine_first(roll4).combine_first(roll3).combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll5], axis=1)
    return df
        
tgs_roll_5 = rolling_tgs_5(data=tgs_clean_5, roll_value=5, roll_type='mean')
tgs_roll_5= tgs_roll_5[~tgs_roll_5['year_overall_pff_5s'].isin(['nan'])]

In [29]:
tgs_roll_5.rename(columns={'unique_team_id_overall_pff_5s':'unique_team_id_overall_pff'}, inplace=True)
tgs_roll_5 = tgs_roll_5[['unique_team_id_overall_pff',
       'pf_overall_pff_5s', 'pa_overall_pff_5s', 'overall_performance_overall_pff_5s',
       'offense_overall_pff_5s', 'pass_overall_pff_5s', 'pass_blocking_overall_pff_5s',
       'receiving_overall_pff_5s', 'rushing_overall_pff_5s',
       'run_blocking_overall_pff_5s', 'defense_overall_pff_5s',
       'rush_defense_overall_pff_5s', 'tackling_overall_pff_5s',
       'pass_rush_overall_pff_5s', 'coverage_overall_pff_5s',
       'special_teams_overall_pff_5s', 'wl_int_overall_pff_5s',
       'fav_homeoraway_overall_pff_5s', 'team_favored_or_not_overall_pff_5s',
       'ats_w_overall_pff_5s', 'ats_l_overall_pff_5s']]

In [30]:
tgs_roll = pd.merge(tgs_roll, tgs_roll_5, on='unique_team_id_overall_pff', how='left')

In [31]:
tgs_roll.drop(['year_overall_pff', 'week_overall_pff', 
                'team_id_impute_overall_pff','team_name_overall_pff'], axis=1, inplace=True)
tgs_roll.rename(columns={'unique_team_id_overall_pff':'unique_team_id'}, inplace=True)
tgs_roll = pd.merge(OU_ids, tgs_roll, on='unique_team_id', how='left')

In [32]:
tgs_roll.columns

Index(['unique_team_id', 'wl', 'pf', 'opponent_id', 'opponent_id_overall_pff',
       'pf_overall_pff', 'pa_overall_pff', 'overall_performance_overall_pff',
       'offense_overall_pff', 'pass_overall_pff', 'pass_blocking_overall_pff',
       'receiving_overall_pff', 'rushing_overall_pff',
       'run_blocking_overall_pff', 'defense_overall_pff',
       'rush_defense_overall_pff', 'tackling_overall_pff',
       'pass_rush_overall_pff', 'coverage_overall_pff',
       'special_teams_overall_pff', 'wl_int_overall_pff',
       'fav_homeoraway_overall_pff', 'team_favored_or_not_overall_pff',
       'ats_w_overall_pff', 'ats_l_overall_pff', 'pf_overall_pff_5s',
       'pa_overall_pff_5s', 'overall_performance_overall_pff_5s',
       'offense_overall_pff_5s', 'pass_overall_pff_5s',
       'pass_blocking_overall_pff_5s', 'receiving_overall_pff_5s',
       'rushing_overall_pff_5s', 'run_blocking_overall_pff_5s',
       'defense_overall_pff_5s', 'rush_defense_overall_pff_5s',
       'tackling_ov

In [33]:
rush_std.head().T

,0,1,2,3,4
player,Chester Taylor,LaDainian Tomlinson,Willie Parker,Warrick Dunn,Rudi Johnson
position,HB,HB,HB,HB,HB
team_name,MIN,SD,PIT,ATL,CIN
player_game_count,1,1,1,1,1
attempts,31,31,29,29,28
avoided_tackles,4,1,4,2,2
breakaway_attempts,0,1,1,3,2
breakaway_percent,0,44.3,27.8,48.5,39.6
breakaway_yards,0,58,32,64,38
declined_penalties,0,0,0,0,0


### Clean and Subset PFF Data into Positional Groupings

In [34]:
##  clean the player name field from pfr function above
qb_standard['player'] = qb_standard.apply(lambda df: clean_name(df), axis=1)
oline['player'] = oline.apply(lambda df: clean_name(df), axis=1)
rush_std['player'] = rush_std.apply(lambda df: clean_name(df), axis=1)
rec_std['player'] = rec_std.apply(lambda df: clean_name(df), axis=1)
def_standard['player'] = def_standard.apply(lambda df: clean_name(df), axis=1)


##  functions to clean and scrub data by source  ##
def clean_pff(df):
##  basic scrubbing to clean data ##
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)

   
    ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(cleaning_dicts.pos_dict).fillna(df['position'])

   
    ##  create our unique ids  ##
    df.insert(0, "unique_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "player_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(2, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(4, "player_name", (df['player']))
    df.insert(5, "player_team", (df['player']+'_'+df['team_name']))   
#    df['year'] = df['year'].astype(int)
#    df['week'] = df['week'].astype(int)
    df = df.apply(pd.to_numeric, errors='ignore')
   
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_id_impute'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
    #df = df.groupby('year').transform(lambda x: (x - x.mean()) / x.std())
    
    return df



rush_std = clean_pff(rush_std)
qb_standard =clean_pff(qb_standard)
oline=clean_pff(oline)
rec_std=clean_pff(rec_std)
def_standard=clean_pff(def_standard)


In [35]:
qb_standard.tail().T

,9545,9546,9547,9548,9549
unique_id,joeflacco_nyj_2021_10,taysomhill_no_2021_10,ezekielelliott_dal_2021_10,tommytownsend_kc_2021_10,johnnyhekker_lar_2021_10
player_id,joeflacco_nyj_2021,taysomhill_no_2021,ezekielelliott_dal_2021,tommytownsend_kc_2021,johnnyhekker_lar_2021
unique_team_id,nyj_2021_10,no_2021_10,dal_2021_10,kc_2021_10,lar_2021_10
team_id_impute,nyj_2021,no_2021,dal_2021,kc_2021,lar_2021
player_name,joeflacco,taysomhill,ezekielelliott,tommytownsend,johnnyhekker
player_team,joeflacco_nyj,taysomhill_no,ezekielelliott_dal,tommytownsend_kc,johnnyhekker_lar
player,joeflacco,taysomhill,ezekielelliott,tommytownsend,johnnyhekker
position,qb,qb,hb,p,p
team_name,nyj,no,dal,kc,lar
player_game_count,1,1,1,1,1


In [36]:
spread_comb.columns

Index(['home_matchup_id', 'away_matchup_id', 'team_id', 'schedule_date',
       'schedule_season', 'schedule_week', 'team_home_abb', 'score_home',
       'score_away', 'away_team_abb', 'team_favorite_id', 'spread_favorite',
       'over_under_line', 'fav_spread', 'team_notfav_id', 'fav_cover',
       'over_under_result', 'precip', 'dome', 'temperature', 'wind_mph',
       'team_abb', 'fav_homeoraway', 'team_ids', 'team_conference',
       'team_division', 'team', 'w-l', 'year', 'week', 'off_dvoa_rank',
       'off_dvoa', 'off_weight_dvoa_rank', 'off_weight_dvoa', 'pass_off_rank',
       'pass_off', 'rush_off_rank', 'rush_off', 'def_dvoa_rank', 'def_dvoa',
       'def_weight_dvoa_rank', 'def_weight_dvoa', 'pass_def_rank', 'pass_def',
       'rush_def_rank', 'rush_def', 'off_dvoa_momentum',
       'off_wt_dvoa_momentum', 'pass_off_dvoa_momentum',
       'rush_off_dvoa_momentum', 'def_dvoa_momentum', 'def_wt_dvoa_momentum',
       'pass_def_dvoa_momentum', 'rush_dvoa_momentum'],
      dty

In [37]:
spread_comb2 = spread_comb[['home_matchup_id', 'away_matchup_id', 'team_id', 'schedule_season', 'schedule_week', 'team_home_abb', 'score_home',
       'score_away', 'away_team_abb', 'team_favorite_id', 'spread_favorite',
       'over_under_line', 'fav_spread', 'team_notfav_id',]]

In [38]:
fo.head().T

,3,56,76,110,142
team_id,ari_2006_1,ari_2006_2,ari_2006_3,ari_2006_4,ari_2006_5
year,2006,2006,2006,2006,2006
week,1,2,3,4,5
team,ari,ari,ari,ari,ari
w-l,1-0,1-1,1-2,1-3,1-4
off_dvoa_rank,8,17,26,30,30
off_dvoa,19.3,-6.7,-13.6,-28.7,-23.6
off_weight_dvoa_rank,8,17,26,30,30
off_weight_dvoa,19.3,-6.7,-13.6,-28.7,-23.6
pass_off_rank,7,14,23,28,27


import numpy as np
import pandas as pd

#fo = pd.read_csv('U:\\Data\\firefox\\gov\\fo_final.csv', error_bad_lines=True)
#qb = pd.read_csv('//ain2/dfsroot/userdata21/thomapb4/Data/Desktop Backup/FP.txt', sep=',', error_bad_lines=True)
qb = qb_standard
match=spread_comb2

#match = pd.read_csv('U:\\Data\\firefox\\gov\\predssub.csv', error_bad_lines=True)
match['isin_team_id'] = match.apply(lambda x: x.team_favorite_id in x.team_id, axis=1)

def hora(nData):
    if nData['isin_team_id'] == True:
        return nData['team_notfav_id']
    else:
        return nData['team_favorite_id']

match['opp_abb'] = match.apply(lambda nData: hora(nData), axis=1)


match['isin_team_id'] = match.apply(lambda x: x.team_home_abb in x.team_id, axis=1)

def scora(nData):
    if nData['isin_team_id'] == True:
        return nData['score_home']
    else:
        return nData['score_away']

match['score'] = match.apply(lambda nData: scora(nData), axis=1)


match['schedule_season']=match['schedule_season'].apply(str)
match['schedule_week']=match['schedule_week'].apply(str)
match['opp_id']= match.opp_abb.str.cat(match.schedule_season, sep="_").str.cat(match.schedule_week, sep="_")
match = match[['team_id','opp_id','score']]

qb = pd.merge(qb, match, left_on='team_id_impute', right_on='team_id', how='left').merge(fo, left_on='opp_id', right_on='team_id', how='left')

#m = pd.merge(qb, fo, on='team_id', how='left')
m = qb[['unique_id','player_team','team_id_x','player','year_x','week_x','score','pass_pff_off','pass_pff_pass', 'passes_underpressure', 'pass_time_to_throw','pass_adj_comp_perc','nfl_passer_rating','def_dvoa_rank','pass_def_rank']]
m['tough_matchup'] = np.where(m['def_dvoa_rank'] <= 15, 'tough', 'easy')


conditions = [m['def_dvoa_rank'] <= 10, m['def_dvoa_rank'] >= 21]
choices = ['hard', 'easy']

m['tough_matchup'] = np.select(conditions, choices)
m['tough_matchup'] = m['tough_matchup'].str.replace('0','mid')
grp = m.groupby(['player','tough_matchup']).mean().reset_index()
grp2 = m.groupby(['player','tough_matchup']).agg({'unique_id':'count'}).reset_index()
grp2.drop(['player','tough_matchup'], axis=1, inplace=True)
grp2.columns = ['game_exp']
grp = pd.concat([grp, grp2], axis=1)

inexp = grp[(grp['game_exp'] <= 5)]
inexp = inexp.groupby(['tough_matchup']).mean().reset_index()

ls = []
for yr in range(2007, 2021, 1):
    for wk in range(1, 18, 1):
        print(yr, wk)
        temp = m[(m['year_x'] <= int(yr)) & (m['year_x'] >= int(yr) - 5)]
        trash = temp[(temp['year_x'] == int(yr)) & (temp['week_x'] >= int(wk))]
        temp=temp[~temp.index.isin(trash.index)]
        temp.drop_duplicates(subset=['player','year_x','week_x'], keep='first', inplace=True)

        conditions = [temp['def_dvoa_rank'] <= 10, temp['def_dvoa_rank'] >= 21]
        choices = ['hard', 'easy']
        
        temp['tough_matchup'] = np.select(conditions, choices)
        temp['tough_matchup'] = temp['tough_matchup'].str.replace('0','mid')
        grp = temp.groupby(['player','tough_matchup']).mean().reset_index()
        grp2 = temp.groupby(['player','tough_matchup']).agg({'unique_id':'count'}).reset_index()
        grp2.drop(['player','tough_matchup'], axis=1, inplace=True)
        grp2.columns = ['game_exp']
        grp = pd.concat([grp, grp2], axis=1)
        grp['year_x'] = str(yr)
        grp['week_x'] = str(wk)
        #grp['player_grp_id']= grp['player'].str.cat(str(yr), sep="_").str.cat(str(wk), sep="_")
        ls.append(grp)

matchups = pd.concat(ls)
matchups['unique__id']= matchups.player.str.cat(matchups.year_x, sep="_").str.cat(matchups.week_x, sep="_")

### Create copies after the groupby imputes b/c it take time

In [39]:
rush_std_copy = rush_std
qb_standard_copy= qb_standard
oline_copy=oline
rec_std_copy=rec_std
def_standard_copy=def_standard

rush_std = rush_std_copy
qb_standard = qb_standard_copy
oline=oline_copy
rec_std=rec_std_copy
def_standard=def_standard_copy

In [40]:
rush_std = rush_std_copy
qb_standard = qb_standard_copy
oline=oline_copy
rec_std=rec_std_copy
def_standard=def_standard_copy

In [41]:
#This cell is foor exporting the player pool to for spot testing purposes#
#rush_std_new.to_csv(cur_dir+'/other_data/rush_std_new_pp.csv')
#qb_standard_new.to_csv(cur_dir+'/other_data/qb_standard_conc.csv')
#oline_standard_new.to_csv(cur_dir+'/other_data/oline_standard_new_pp.csv')
#rec_std_new.to_csv(cur_dir+'/other_data/rec_std_new_pp.csv')
#def_standard_new.to_csv(cur_dir+'/other_data/def_standard_new_pp.csv')

Combine injury data onto clean pff datasets for current week

In [42]:
rush_std_new = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/rush_std_new_pp_w'+cur_week+'.csv')
qb_standard_new = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/qb_standard_new_pp_w'+cur_week+'.csv')
oline_standard_new = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/oline_standard_new_pp_w'+cur_week+'.csv')
rec_std_new = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/rec_std_new_pp_w'+cur_week+'.csv')
def_standard_new = pd.read_csv(cur_dir+"/current_season_data/"+cur_week_folder+'/def_standard_new_pp_w'+cur_week+'.csv')

rush_std_new['week'] = rush_std_new['plyr_number']
qb_standard_new['week'] = qb_standard_new['plyr_number']
oline_standard_new['week'] = oline_standard_new['plyr_number']
rec_std_new['week'] = rec_std_new['plyr_number']
def_standard_new['week'] = def_standard_new['plyr_number']

In [43]:
qb_standard_new.drop('plyr_number', axis=1, inplace=True)
oline_standard_new.drop('plyr_number', axis=1, inplace=True)
rush_std_new.drop('plyr_number', axis=1, inplace=True)
rec_std_new.drop('plyr_number', axis=1, inplace=True)
def_standard_new.drop('plyr_number', axis=1, inplace=True)

In [44]:
qb_standard = pd.concat([qb_standard, qb_standard_new], axis=0)
oline = pd.concat([oline, oline_standard_new], axis=0)
#qb_adj = pd.concat([qb_adj, qb_adj_2019, qb_adj_nw], axis=0)
rush_std = pd.concat([rush_std, rush_std_new], axis=0)
rec_std = pd.concat([rec_std, rec_std_new], axis=0)
def_standard = pd.concat([def_standard, def_standard_new], axis=0)

#qb_adj = qb_adj[['unique_id','qb_adjform','adj_perc']]
#qb_standard = pd.merge(qb_standard, qb_adj, on='unique_id', how='left')


rush_std=rush_std.apply(pd.to_numeric, errors='ignore')
def_rbs = rush_std[rush_std['position'].isin(['hb','fb','qb'])]

qb_standard=qb_standard.apply(pd.to_numeric, errors='ignore')
def_qbs = qb_standard[qb_standard['position'].isin(['qb'])]

rec_std=rec_std.apply(pd.to_numeric, errors='ignore')
def_rec = rec_std[rec_std['position'].isin(['wr','te','hb'])]

#oline['position']=oline['oline_starter']
oline=oline.apply(pd.to_numeric, errors='ignore')
def_oline = oline[oline['position'].isin(['t','g','ol','c'])]

defense=def_standard.apply(pd.to_numeric, errors='ignore')

In [45]:

def_oline.position.value_counts()

t     19637
g     19201
ol     8934
Name: position, dtype: int64

### Roll Data Function

In [46]:


def rolling(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["player","team_name","year","week"], ascending=[True, True, True, True])
    #data=data.fillna(data.mean())
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    elif roll_type == 'std':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).std())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).std())
        roll3 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).std())
        roll2 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).std())
        roll1 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-3).std())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    elif roll_type == 'var':
        roll3 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).var())
        roll2 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).var())
        roll1 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).var())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
   
roll_rbs = rolling(data=def_rbs, roll_value=3, roll_type='mean')
roll_qbs = rolling(data=def_qbs, roll_value=3, roll_type='mean')
roll_wr_recs = rolling(data= def_rec, roll_value=3, roll_type='mean')
roll_oline = rolling(data=def_oline, roll_value=3, roll_type='mean')
roll_defense = rolling(data=defense, roll_value=3, roll_type='mean')


In [47]:
rbs=roll_rbs
qbs=roll_qbs
wrs=roll_wr_recs
ol=roll_oline
defense=roll_defense

In [123]:
roll_rbs=rbs
roll_qbs=qbs
roll_wr_recs=wrs
roll_oline=ol
roll_defense=defense

In [124]:
roll_rbs.to_csv(cur_dir+'/other_data/roll_rbs.csv')

In [125]:
num_cols = def_rbs.select_dtypes(include=[np.number]).columns.tolist()
print(num_cols)

['player_game_count', 'attempts', 'avoided_tackles', 'breakaway_attempts', 'breakaway_percent', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 'elusive_rating', 'explosive', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_offense', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run', 'grades_run_block', 'longest', 'penalties', 'rec_yards', 'receptions', 'routes', 'run_plays', 'scramble_yards', 'scrambles', 'targets', 'total_touches', 'touchdowns', 'yards', 'yards_after_contact', 'yco_attempt', 'ypa', 'yprr', 'zone_attempts', 'week', 'year']


In [126]:
def_oline.T

,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
unique_id,peneisewell_det_2006_1,halapoulivaativaitai_det_2006_1,frankragnow_det_2006_1,mattnelson_det_2006_1,jonahjackson_det_2006_1,koltonmiller_lv_2006_1,johnsimpson_lv_2006_1,alexleatherwood_lv_2006_1,andrejames_lv_2006_1,jonfeliciano_buf_2006_1,...,davidquessenberry_ten_2021_11,taylorlewan_ten_2021_11,aaronbrewer_ten_2021_11,kendalllamm_ten_2021_11,charleslenojr_was_2021_11,ereckflowers_was_2021_11,tylerlarsen_was_2021_11,corneliuslucas_was_2021_11,brandonscherff_was_2021_11,wesschweitzer_was_2021_11
player_id,peneisewell_det_2006,halapoulivaativaitai_det_2006,frankragnow_det_2006,mattnelson_det_2006,jonahjackson_det_2006,koltonmiller_lv_2006,johnsimpson_lv_2006,alexleatherwood_lv_2006,andrejames_lv_2006,jonfeliciano_buf_2006,...,davidquessenberry_ten_2021,taylorlewan_ten_2021,aaronbrewer_ten_2021,kendalllamm_ten_2021,charleslenojr_was_2021,ereckflowers_was_2021,tylerlarsen_was_2021,corneliuslucas_was_2021,brandonscherff_was_2021,wesschweitzer_was_2021
unique_team_id,det_2006_1,det_2006_1,det_2006_1,det_2006_1,det_2006_1,lv_2006_1,lv_2006_1,lv_2006_1,lv_2006_1,buf_2006_1,...,ten_2021_11,ten_2021_11,ten_2021_11,ten_2021_11,was_2021_11,was_2021_11,was_2021_11,was_2021_11,was_2021_11,was_2021_11
team_id_impute,det_2006,det_2006,det_2006,det_2006,det_2006,lv_2006,lv_2006,lv_2006,lv_2006,buf_2006,...,ten_2021,ten_2021,ten_2021,ten_2021,was_2021,was_2021,was_2021,was_2021,was_2021,was_2021
player_name,peneisewell,halapoulivaativaitai,frankragnow,mattnelson,jonahjackson,koltonmiller,johnsimpson,alexleatherwood,andrejames,jonfeliciano,...,davidquessenberry,taylorlewan,aaronbrewer,kendalllamm,charleslenojr,ereckflowers,tylerlarsen,corneliuslucas,brandonscherff,wesschweitzer
player_team,peneisewell_det,halapoulivaativaitai_det,frankragnow_det,mattnelson_det,jonahjackson_det,koltonmiller_lv,johnsimpson_lv,alexleatherwood_lv,andrejames_lv,jonfeliciano_buf,...,davidquessenberry_ten,taylorlewan_ten,aaronbrewer_ten,kendalllamm_ten,charleslenojr_was,ereckflowers_was,tylerlarsen_was,corneliuslucas_was,brandonscherff_was,wesschweitzer_was
player,peneisewell,halapoulivaativaitai,frankragnow,mattnelson,jonahjackson,koltonmiller,johnsimpson,alexleatherwood,andrejames,jonfeliciano,...,davidquessenberry,taylorlewan,aaronbrewer,kendalllamm,charleslenojr,ereckflowers,tylerlarsen,corneliuslucas,brandonscherff,wesschweitzer
position,t,g,ol,t,g,t,g,t,ol,g,...,t,t,g,t,t,g,ol,t,g,g
team_name,det,det,det,det,det,lv,lv,lv,lv,buf,...,ten,ten,ten,ten,was,was,was,was,was,was
player_game_count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [127]:
## Get rid of nonsense ##
roll_rbs= roll_rbs[~roll_rbs['year'].isin(['nan'])]
roll_qbs= roll_qbs[~roll_qbs['year'].isin(['nan'])]
roll_wr_recs= roll_wr_recs[~roll_wr_recs['year'].isin(['nan'])]
roll_oline= roll_oline[~roll_oline['year'].isin(['nan'])]
roll_defense= roll_defense[~roll_defense['year'].isin(['nan'])]

## Subset into defense positional groups ##
def_lineman = roll_defense[roll_defense['position'].isin(['ed','di'])]
def_linebackers = roll_defense[roll_defense['position'].isin(['lb'])]
def_dbs = roll_defense[roll_defense['position'].isin(['cb','s'])]

In [128]:
roll_qbs_rush = roll_rbs[roll_rbs['position'].isin(['qb'])]
roll_rbs = roll_rbs[~roll_rbs.index.isin(roll_qbs_rush.index)]


roll_te_rec = roll_wr_recs[roll_wr_recs['position'].isin(['te'])]
#roll_wr_recs = roll_wr_recs[~roll_wr_recs.index.isin(roll_te_rec.index)]

roll_rb_rec = roll_wr_recs[roll_wr_recs['position'].isin(['hb'])]
#roll_wr_recs = roll_wr_recs[~roll_wr_recs.index.isin(roll_rb_rec.index)]

roll_wr_recs = roll_wr_recs[roll_wr_recs['position'].isin(['wr'])]

In [129]:
profiles = pd.read_csv(cur_dir+'/other_data/pff_player_pros.csv')
profiles.head().T
rb_df = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
te_df = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
qb_df = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
wr_df = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
ol_df = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
def_dl = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
def_lb = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]
def_db = profiles[['unique_id','height','wt','speed','draft_yr','round','selection']]

In [130]:
roll_qbs

,unique_id,player_id,unique_team_id,team_id_impute,player_name,player_team,player,position,team_name,player_game_count,...,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa,week,year
1,aaronbrooks_oak_2006_11,aaronbrooks_oak_2006,oak_2006_11,oak_2006,aaronbrooks,aaronbrooks_oak,aaronbrooks,qb,oak,1.0,...,3.000000,0.000000,1.000000,0.000000,2.000000,8.000000,68.000000,4.900000,1.0,2006.0
2,aaronbrooks_oak_2006_12,aaronbrooks_oak_2006,oak_2006_12,oak_2006,aaronbrooks,aaronbrooks_oak,aaronbrooks,qb,oak,1.0,...,3.500000,0.000000,2.000000,0.500000,1.500000,5.700000,123.500000,6.500000,6.0,2006.0
3,aaronbrooks_oak_2006_13,aaronbrooks_oak_2006,oak_2006_13,oak_2006,aaronbrooks,aaronbrooks_oak,aaronbrooks,qb,oak,1.0,...,3.333333,0.000000,1.666667,0.666667,1.666667,5.400000,144.666667,6.400000,8.0,2006.0
4,aaronbrooks_oak_2006_14,aaronbrooks_oak_2006,oak_2006_14,oak_2006,aaronbrooks,aaronbrooks_oak,aaronbrooks,qb,oak,1.0,...,3.000000,0.666667,2.000000,0.666667,1.666667,4.033333,201.333333,6.666667,12.0,2006.0
5,aaronbrooks_oak_2006_15,aaronbrooks_oak_2006,oak_2006_15,oak_2006,aaronbrooks,aaronbrooks_oak,aaronbrooks,qb,oak,1.0,...,2.333333,0.666667,1.333333,0.666667,1.333333,2.900000,208.000000,5.666667,13.0,2006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9126,zachwilson_nyj_2021_2,zachwilson_nyj_2021,nyj_2021_2,nyj_2021,zachwilson,zachwilson_nyj,zachwilson,qb,nyj,1.0,...,0.000000,0.000000,1.000000,2.000000,3.000000,6.500000,258.000000,7.000000,1.0,2021.0
9127,zachwilson_nyj_2021_3,zachwilson_nyj_2021,nyj_2021_3,nyj_2021,zachwilson,zachwilson_nyj,zachwilson,qb,nyj,1.0,...,1.000000,0.000000,0.500000,1.000000,2.500000,5.600000,234.000000,6.700000,1.5,2021.0
9128,zachwilson_nyj_2021_4,zachwilson_nyj_2021,nyj_2021_4,nyj_2021,zachwilson,zachwilson_nyj,zachwilson,qb,nyj,1.0,...,0.666667,0.000000,1.000000,0.666667,2.000000,4.533333,209.333333,6.000000,2.0,2021.0
9129,zachwilson_nyj_2021_5,zachwilson_nyj_2021,nyj_2021_5,nyj_2021,zachwilson,zachwilson_nyj,zachwilson,qb,nyj,1.0,...,1.000000,0.000000,1.000000,0.666667,2.000000,5.000000,222.333333,6.566667,3.0,2021.0


### RB Position Weights and Stats Groupby's

In [131]:
roll_rbs.columns

Index(['unique_id', 'player_id', 'unique_team_id', 'team_id_impute',
       'player_name', 'player_team', 'player', 'position', 'team_name',
       'player_game_count', 'attempts', 'avoided_tackles',
       'breakaway_attempts', 'breakaway_percent', 'breakaway_yards',
       'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf',
       'elu_rush_mtf', 'elu_yco', 'elusive_rating', 'explosive', 'first_downs',
       'franchise_id', 'fumbles', 'gap_attempts', 'grades_offense',
       'grades_offense_penalty', 'grades_pass', 'grades_pass_block',
       'grades_pass_route', 'grades_run', 'grades_run_block', 'longest',
       'penalties', 'rec_yards', 'receptions', 'routes', 'run_plays',
       'scramble_yards', 'scrambles', 'targets', 'total_touches', 'touchdowns',
       'yards', 'yards_after_contact', 'yco_attempt', 'ypa', 'yprr',
       'zone_attempts', 'week', 'year'],
      dtype='object')

In [57]:
## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
rb_df.columns = [str(col) + '_rbs' for col in rb_df.columns]
roll_rbs = pd.merge(roll_rbs, rb_df, left_on='player_id', right_on='unique_id_rbs', how='left')
roll_rbs.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
def rush_att(nData):
    if nData['attempts'] == 0:
        return 1
    else:
        return nData['attempts']
roll_rbs['attempts'] = roll_rbs.apply(lambda df: rush_att(df), axis=1)  
roll_qbs_rush['attempts'] = roll_qbs_rush.apply(lambda df: rush_att(df), axis=1)

def game_combine(data=None):
    data_cols = data.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex=r'grades_|week|year|team_id|height|wt|speed|draft_yr|round|selection|franchise')))].columns.tolist()
    group = data.groupby(data['unique_team_id'])[num_cols].mean().reset_index()
    return group
   
rb_sums = game_combine(data=roll_rbs)
qbs_rush_sums = game_combine(data=roll_qbs_rush)


def weighted(nData, snap_Var='attempts'):
    pff_cols = nData.filter(regex='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
   
def pros(nData, snap_Var='attempts'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)

roll_rbs_avgs=roll_rbs.fillna(roll_rbs.mean())
rb_pro_avgs = roll_rbs_avgs.groupby('unique_team_id').apply(pros).reset_index()

roll_rbs_zeros=roll_rbs.fillna(0.1)
roll_qbs_zeros=roll_qbs_rush.fillna(0.1)

rb_pff_avgs = roll_rbs_zeros.groupby('unique_team_id').apply(weighted).reset_index()
qb_rush_pff_avgs = roll_qbs_zeros.groupby('unique_team_id').apply(weighted).reset_index()

rb_stats = pd.merge(rb_pff_avgs, rb_pro_avgs, on='unique_team_id', how='left').merge(rb_sums, on='unique_team_id', how='left')
rb_stats.head(n=2).T


qb_rush_stats = pd.merge(qb_rush_pff_avgs, qbs_rush_sums, on='unique_team_id', how='left')


In [133]:
qb_rush_stats = qb_rush_stats.add_suffix('_qb_rush')
qb_rush_stats.rename(columns={'unique_team_id_qb_rush':'unique_team_id'}, inplace=True)
qb_rush_stats

,unique_team_id,grades_offense_qb_rush,grades_offense_penalty_qb_rush,grades_pass_qb_rush,grades_pass_block_qb_rush,grades_pass_route_qb_rush,grades_run_qb_rush,grades_run_block_qb_rush,player_game_count_qb_rush,attempts_qb_rush,...,scrambles_qb_rush,targets_qb_rush,total_touches_qb_rush,touchdowns_qb_rush,yards_qb_rush,yards_after_contact_qb_rush,yco_attempt_qb_rush,ypa_qb_rush,yprr_qb_rush,zone_attempts_qb_rush
0,ari_2006_10,53.766667,26.400000,56.400000,64.947329,60.983973,54.100000,60.0,1.0,0.666667,...,1.000000,0.000000,0.666667,0.000000,3.666667,3.000000,1.890000,2.566667,1.244215,0.000000
1,ari_2006_11,49.033333,26.300000,50.500000,64.947329,60.983973,58.066667,60.0,1.0,1.666667,...,1.000000,0.000000,1.666667,0.333333,8.333333,2.333333,0.973333,2.733333,1.244215,0.333333
2,ari_2006_12,58.600000,52.533333,59.800000,64.947329,60.983973,59.266667,60.0,1.0,2.333333,...,1.333333,0.000000,2.333333,0.666667,7.000000,2.333333,0.693333,2.233333,1.244215,0.333333
3,ari_2006_14,66.433333,52.533333,66.633333,64.947329,60.983973,60.535054,60.0,1.0,2.333333,...,1.333333,0.000000,2.333333,0.666667,5.000000,2.000000,0.360000,0.233333,1.244215,0.333333
4,ari_2006_15,67.233333,52.633333,68.833333,64.947329,60.983973,55.035054,60.0,1.0,2.000000,...,1.000000,0.000000,2.000000,0.333333,-2.000000,1.666667,0.276667,-1.766667,1.244215,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6303,was_2021_3,60.150000,78.250000,62.700000,64.947329,60.983973,50.350000,60.0,1.0,0.500000,...,3.000000,0.000000,0.500000,0.000000,11.500000,9.000000,2.375000,3.600000,1.244215,0.000000
6304,was_2021_4,53.366667,52.166667,55.500000,64.947329,60.983973,52.366667,60.0,1.0,1.000000,...,4.000000,0.333333,1.333333,0.333333,14.666667,6.000000,1.583333,3.266667,1.244215,0.000000
6305,was_2021_5,60.033333,52.833333,59.800000,64.947329,60.983973,59.333333,60.0,1.0,1.333333,...,4.333333,0.333333,1.666667,0.333333,23.333333,7.666667,1.783333,4.233333,1.244215,0.000000
6306,was_2021_7,52.000000,53.033333,49.766667,64.947329,60.983973,66.566667,60.0,1.0,1.000000,...,5.000000,0.333333,1.333333,0.333333,34.666667,4.000000,0.800000,6.400000,1.244215,0.000000


In [58]:
roll_rbs.groupby('unique_team_id').apply(weighted).reset_index()

,unique_team_id,grades_offense,grades_offense_penalty,grades_pass,grades_pass_block,grades_pass_route,grades_run,grades_run_block
0,ari_2006_10,46.565432,74.860905,64.863132,80.397377,60.093645,44.837860,60.430778
1,ari_2006_11,52.180303,75.333333,64.863132,80.797351,62.519563,48.923333,60.210851
2,ari_2006_12,60.490104,75.264583,64.863132,78.083333,64.738021,58.083854,60.224496
3,ari_2006_13,61.080556,73.931250,64.863132,73.945430,65.206611,58.175694,60.076970
4,ari_2006_14,57.584742,73.977230,64.863132,72.923709,61.717840,56.220657,60.276080
...,...,...,...,...,...,...,...,...
7492,was_2021_4,59.713559,56.471186,64.863132,54.408371,53.186441,63.640678,60.214649
7493,was_2021_5,65.678531,75.955367,64.863132,44.012011,60.444633,67.307345,60.062620
7494,was_2021_6,68.162626,75.502020,64.863132,62.263528,60.412484,68.074747,60.038274
7495,was_2021_7,62.285792,76.363388,64.863132,56.797814,57.933333,62.778142,60.092314


In [59]:
rb_stats.to_csv(cur_dir+'/other_data/roll_rbs.csv')

In [60]:
roll_qbs_zeros.head().T

,1,2,3,4,5
unique_id,aaronbrooks_oak_2006_2,aaronbrooks_oak_2006_11,aaronbrooks_oak_2006_12,aaronbrooks_oak_2006_13,aaronbrooks_oak_2006_14
player_id,aaronbrooks_oak_2006,aaronbrooks_oak_2006,aaronbrooks_oak_2006,aaronbrooks_oak_2006,aaronbrooks_oak_2006
unique_team_id,oak_2006_2,oak_2006_11,oak_2006_12,oak_2006_13,oak_2006_14
team_id_impute,oak_2006,oak_2006,oak_2006,oak_2006,oak_2006
player_name,aaronbrooks,aaronbrooks,aaronbrooks,aaronbrooks,aaronbrooks
player_team,aaronbrooks_oak,aaronbrooks_oak,aaronbrooks_oak,aaronbrooks_oak,aaronbrooks_oak
player,aaronbrooks,aaronbrooks,aaronbrooks,aaronbrooks,aaronbrooks
position,qb,qb,qb,qb,qb
team_name,oak,oak,oak,oak,oak
player_game_count,1,1,1,1,1


### QB Position Weights and Stats Groupby's

In [61]:

qb_df.columns = [str(col) + '_qbs' for col in qb_df.columns]
roll_qbs = pd.merge(roll_qbs, qb_df, left_on='player_id', right_on='unique_id_qbs', how='left')
roll_qbs.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

def pass_att(nData):
    if nData['dropbacks'] == 0:
        return 1
    else:
        return nData['dropbacks']
roll_qbs['dropbacks'] = roll_qbs.apply(lambda df: pass_att(df), axis=1)   

def weighted(nData, snap_Var='dropbacks'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)

def pros(nData, snap_Var='dropbacks'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)

roll_qbs_avgs=roll_qbs.fillna(roll_qbs.mean())
qb_pro_avgs = roll_qbs_avgs.groupby('unique_team_id').apply(pros).reset_index()

roll_qbs_zeros=roll_qbs.fillna(0.1)
qb_pff_avgs = roll_qbs_zeros.groupby('unique_team_id').apply(weighted).reset_index()

qb_sums = game_combine(data=roll_qbs)

qb_stats = pd.merge(qb_pff_avgs, qb_pro_avgs, on='unique_team_id', how='left').merge(qb_sums, on='unique_team_id', how='left')
qb_stats.head().T


,0,1,2,3,4
unique_team_id,ari_2006_10,ari_2006_11,ari_2006_12,ari_2006_13,ari_2006_14
grades_hands_fumble,79.2,62.6333,45.7,46.2,62
grades_offense,47.1,49.0333,58.6,66.4333,72.6
grades_pass,49.1667,50.5,59.8,66.6333,73.1667
grades_run,55.2257,58.0667,59.2667,59.8924,57.0515
height_qbs,77,77,77,77,77
wt_qbs,230,230,230,230,230
speed_qbs,4.9,4.9,4.9,4.9,4.9
draft_yr_qbs,2006,2006,2006,2006,2006
round_qbs,1,1,1,1,1


In [62]:
qb_stats.tail().T

,7388,7389,7390,7391,7392
unique_team_id,was_2021_4,was_2021_5,was_2021_6,was_2021_7,was_2021_8
grades_hands_fumble,78.2667,79.6333,63.6333,63.7667,48.1
grades_offense,53.3667,60.0333,52,61.2333,57.7333
grades_pass,55.5,59.8,49.7667,58.2333,55.5
grades_run,52.3667,59.3333,66.5667,67.3257,64.2924
height_qbs,73,73,73,73,73
wt_qbs,205,205,205,205,205
speed_qbs,4.82827,4.82827,4.82827,4.82827,4.82827
draft_yr_qbs,2015,2015,2015,2015,2015
round_qbs,10,10,10,10,10


In [63]:
qb_sums

,unique_team_id,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,avg_time_to_throw,bats,big_time_throws,btt_rate,...,sack_percent,sacks,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa
0,ari_2006_10,1.0,53.466667,33.333333,36.333333,8.833333,2.733333,2.000000,0.666667,1.900000,...,6.900000,2.666667,0.666667,0.000000,0.333333,1.000000,2.333333,5.833333,197.333333,5.433333
1,ari_2006_11,1.0,51.833333,32.666667,35.000000,9.966667,2.890000,0.666667,1.333333,3.533333,...,6.966667,2.666667,1.000000,0.333333,0.333333,0.333333,2.333333,5.833333,192.000000,5.500000
2,ari_2006_12,1.0,61.466667,31.333333,34.000000,9.200000,2.720000,0.666667,2.000000,5.833333,...,5.233333,2.000000,1.333333,0.333333,0.333333,0.666667,1.333333,3.133333,202.000000,6.066667
3,ari_2006_13,1.0,69.600000,36.333333,39.333333,10.266667,2.680000,0.666667,2.666667,6.866667,...,2.433333,1.000000,1.333333,0.666667,0.666667,0.666667,1.333333,2.933333,284.666667,7.200000
4,ari_2006_14,1.0,72.200000,32.666667,34.666667,9.866667,2.596667,0.333333,2.333333,6.466667,...,2.933333,1.000000,1.000000,0.333333,0.666667,1.000000,0.666667,1.800000,274.666667,7.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7388,was_2021_4,1.0,79.433333,27.333333,28.333333,7.766667,3.040000,0.000000,1.000000,2.100000,...,1.733333,0.666667,4.000000,0.333333,0.666667,1.666667,1.000000,2.933333,223.333333,8.066667
7389,was_2021_5,1.0,75.100000,33.666667,34.333333,8.266667,3.083333,0.000000,1.666667,3.966667,...,2.600000,1.000000,4.333333,0.333333,0.333333,2.333333,2.000000,5.300000,279.333333,8.300000
7390,was_2021_6,1.0,65.866667,31.333333,32.666667,9.166667,3.330000,0.666667,1.333333,3.300000,...,3.333333,1.333333,5.000000,0.000000,0.333333,1.666667,3.333333,7.800000,250.000000,7.866667
7391,was_2021_7,1.0,66.333333,35.333333,37.666667,9.100000,2.993333,1.000000,1.333333,3.300000,...,2.266667,1.000000,3.000000,0.000000,1.000000,1.333333,2.333333,4.866667,240.000000,6.500000


### WR Position Weights and Stats Groupby's

In [64]:
roll_wr_recs.columns

Index(['unique_id', 'player_id', 'unique_team_id', 'team_id_impute',
       'player_name', 'player_team', 'player', 'position', 'team_name',
       'player_game_count', 'avg_depth_of_target', 'avoided_tackles',
       'caught_percent', 'contested_catch_rate', 'contested_receptions',
       'contested_targets', 'declined_penalties', 'drop_rate', 'drops',
       'first_downs', 'franchise_id', 'fumbles', 'grades_hands_drop',
       'grades_hands_fumble', 'grades_offense', 'grades_pass_block',
       'grades_pass_route', 'inline_rate', 'inline_snaps', 'interceptions',
       'longest', 'pass_block_rate', 'pass_blocks', 'pass_plays', 'penalties',
       'receptions', 'route_rate', 'routes', 'slot_rate', 'slot_snaps',
       'targeted_qb_rating', 'targets', 'touchdowns', 'wide_rate',
       'wide_snaps', 'yards', 'yards_after_catch',
       'yards_after_catch_per_reception', 'yards_per_reception', 'yprr',
       'week', 'year'],
      dtype='object')

In [65]:

wr_df.columns = [str(col) + '_wrs' for col in wr_df.columns]
roll_wr_recs = pd.merge(roll_wr_recs, wr_df, left_on='player_id', right_on='unique_id_wrs', how='left')
te_df.columns = [str(col) + '_tes' for col in te_df.columns]
roll_te_rec = pd.merge(roll_te_rec, te_df, left_on='player_id', right_on='unique_id_tes', how='left')

roll_wr_recs.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)
roll_te_rec.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)


def pass_att(nData):
   if nData['targets'] == 0:
       return .01
   else:
       return nData['targets']
roll_wr_recs['rec_targets'] = roll_wr_recs.apply(lambda df: pass_att(df), axis=1)   
roll_te_rec['rec_targets'] = roll_te_rec.apply(lambda df: pass_att(df), axis=1) 

def game_pff(data=None):
    pff_cols = data.filter(like='grades_').columns.tolist()
    recrat_cols = data.filter(regex=r'avg_depth_of_target|yprr|yards_|targeted_qb_rating|drop_rate|drops|contested_|caught_|avoid_').columns.tolist()
    pff_cols = pff_cols+recrat_cols
    group = data.groupby(data['unique_team_id'])[pff_cols].mean()
    return group

roll_wr_pffs = game_pff(data=roll_wr_recs)
Fant_colmean = str('_'+'recrollavgs')
roll_wr_pffs.rename(columns=lambda x: x+Fant_colmean, inplace=True)
roll_wr_pffs.reset_index(inplace=True)

roll_te_pffs = game_pff(data=roll_te_rec)
Fant_colmean = str('_'+'te_recrollavgs')
roll_te_pffs.rename(columns=lambda x: x+Fant_colmean, inplace=True)
roll_te_pffs.reset_index(inplace=True)

roll_rb_pffs = game_pff(data=roll_rb_rec)
Fant_colmean = str('_'+'rb_recrollavgs')
roll_rb_pffs.rename(columns=lambda x: x+Fant_colmean, inplace=True)
roll_rb_pffs.reset_index(inplace=True)


def weighted(nData, snap_Var='rec_targets'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)

def pros(nData, snap_Var='rec_targets'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)

roll_wrs_avgs=roll_wr_recs.fillna(roll_wr_recs.mean())
wr_pro_avgs = roll_wrs_avgs.groupby('unique_team_id').apply(pros).reset_index()
roll_te_avgs=roll_te_rec.fillna(roll_te_rec.mean())
te_pro_avgs = roll_te_avgs.groupby('unique_team_id').apply(pros).reset_index()

roll_wrs_zeros=roll_wr_recs.fillna(0.1)
#roll_rbs_zeros=roll_rb_avgs.fillna(0.1)
roll_tes_zeros=roll_te_avgs.fillna(0.1)

wr_pff_targ_avgs = roll_wrs_zeros.groupby('unique_team_id').apply(weighted).reset_index()
te_pff_targ_avgs = roll_tes_zeros.groupby('unique_team_id').apply(weighted).reset_index()
#rb_pff_targ_avgs = roll_rbs_zeros.groupby('unique_team_id').apply(weighted).reset_index()

roll_wr_recs = game_combine(data=roll_wr_recs)
roll_te_rec = game_combine(data=roll_te_rec)
roll_rb_rec = game_combine(data=roll_rb_rec)

wr_stats = pd.merge(roll_wr_pffs, wr_pro_avgs, on='unique_team_id', how='left').merge(wr_pff_targ_avgs, on='unique_team_id', how='left').merge(roll_wr_recs, on='unique_team_id', how='left')
wr_stats.head()


,unique_team_id,grades_hands_drop_recrollavgs,grades_hands_fumble_recrollavgs,grades_offense_recrollavgs,grades_pass_block_recrollavgs,grades_pass_route_recrollavgs,avg_depth_of_target_recrollavgs,caught_percent_recrollavgs,contested_catch_rate_recrollavgs,contested_receptions_recrollavgs,...,targets,touchdowns,wide_rate,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,rec_targets
0,ari_2006_10,65.533333,66.755556,68.533333,64.738077,68.211111,11.100000,55.233333,46.312904,0.0,...,8.000000,0.444444,72.800000,26.000000,56.222222,23.333333,5.655556,13.388889,1.612222,8.000000
1,ari_2006_11,62.883333,65.991667,65.091667,64.738077,64.558333,12.191667,57.050000,46.312904,0.0,...,6.583333,0.166667,60.283333,20.750000,46.833333,13.083333,4.116667,14.816667,1.419167,6.583333
2,ari_2006_12,59.516667,65.483333,66.083333,64.738077,65.591667,11.300000,64.983333,46.312904,0.0,...,5.333333,0.250000,59.408333,19.416667,41.083333,10.666667,3.533333,13.591667,1.425000,5.333333
3,ari_2006_13,61.016667,66.716344,67.991667,64.738077,67.600000,13.733333,61.466667,46.312904,0.0,...,6.916667,0.250000,56.550000,22.250000,62.000000,13.916667,3.324804,14.176434,1.675833,6.916667
4,ari_2006_14,59.066667,66.083011,67.650000,64.738077,67.616667,12.833333,66.600000,46.312904,0.0,...,6.000000,0.250000,52.458333,19.250000,54.500000,11.916667,2.983138,13.059767,1.727500,6.000000


In [66]:
te_stats = pd.merge(roll_te_pffs, te_pro_avgs, on='unique_team_id', how='left').merge(te_pff_targ_avgs, on='unique_team_id', how='left').merge(roll_te_rec, on='unique_team_id', how='left')
te_stats.head()


,unique_team_id,grades_hands_drop_te_recrollavgs,grades_hands_fumble_te_recrollavgs,grades_offense_te_recrollavgs,grades_pass_block_te_recrollavgs,grades_pass_route_te_recrollavgs,avg_depth_of_target_te_recrollavgs,caught_percent_te_recrollavgs,contested_catch_rate_te_recrollavgs,contested_receptions_te_recrollavgs,...,targets,touchdowns,wide_rate,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,rec_targets
0,ari_2006_10,56.777072,63.530751,55.550000,63.029359,49.466667,3.500000,25.000000,46.312904,0.0,...,1.166667,0.0,0.716667,0.166667,5.166667,2.833333,6.498434,12.578136,0.480000,1.166667
1,ari_2006_11,58.711971,64.081397,53.466667,49.896026,51.933333,3.833333,41.666667,46.312904,0.0,...,1.833333,0.0,1.383333,0.333333,9.833333,5.000000,5.748825,11.391935,0.546667,1.833333
2,ari_2006_12,60.281768,63.382688,56.350000,53.400000,53.116667,3.000000,75.000000,46.312904,0.0,...,2.000000,0.0,2.333333,0.333333,11.833333,6.000000,4.582942,8.852867,0.815000,2.000000
3,ari_2006_13,62.000000,54.916667,51.350000,44.433333,52.450000,3.000000,95.833333,46.312904,0.0,...,2.333333,0.0,2.333333,0.333333,13.833333,5.000000,2.216667,5.450000,0.961667,2.333333
4,ari_2006_14,60.665101,54.832688,54.066667,63.756346,53.150000,3.916667,79.166667,46.312904,0.0,...,1.833333,0.0,5.833333,0.333333,11.000000,3.166667,2.549608,6.802867,0.978333,1.833333


In [67]:
rb_rec_stats = pd.merge(roll_rb_pffs, roll_rb_rec, on='unique_team_id', how='left')
rb_rec_stats.head()

,unique_team_id,grades_hands_drop_rb_recrollavgs,grades_hands_fumble_rb_recrollavgs,grades_offense_rb_recrollavgs,grades_pass_block_rb_recrollavgs,grades_pass_route_rb_recrollavgs,avg_depth_of_target_rb_recrollavgs,caught_percent_rb_recrollavgs,contested_catch_rate_rb_recrollavgs,contested_receptions_rb_recrollavgs,...,targeted_qb_rating,targets,touchdowns,wide_rate,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr
0,ari_2006_10,55.938889,63.682257,51.811111,69.528034,55.205556,0.722222,59.255556,46.312904,0.0,...,68.100000,2.000000,0.0,0.00,0.000000,7.833333,6.333333,4.716014,7.399223,1.453333
1,ari_2006_11,62.466667,62.777347,55.300000,64.308462,62.555556,1.466667,62.955556,46.312904,0.0,...,66.900000,2.666667,0.0,0.00,0.000000,9.333333,8.000000,4.799739,6.435245,1.542222
2,ari_2006_12,66.981768,62.382688,59.050000,66.950000,61.483333,1.450000,58.333333,46.312904,0.0,...,64.933333,3.166667,0.0,0.00,0.000000,10.666667,9.666667,5.449608,6.819534,0.825000
3,ari_2006_13,68.300000,48.833333,61.333333,77.366667,64.500000,2.300000,66.666667,46.312904,0.0,...,69.933333,3.666667,0.0,1.50,0.333333,12.333333,9.000000,4.500000,6.166667,0.680000
4,ari_2006_14,64.930203,65.282688,57.516667,65.389680,58.533333,0.366667,61.116667,46.312904,0.0,...,68.183333,1.500000,0.0,0.75,0.166667,5.166667,4.166667,4.749217,6.705735,0.398333


In [68]:
roll_oline.head().T

,1,2,3,4,5
unique_id,aaronbrewer_ten_2020_5,aaronbrewer_ten_2020_7,aaronbrewer_ten_2020_10,aaronbrewer_ten_2020_11,aaronbrewer_ten_2020_12
player_id,aaronbrewer_ten_2020,aaronbrewer_ten_2020,aaronbrewer_ten_2020,aaronbrewer_ten_2020,aaronbrewer_ten_2020
unique_team_id,ten_2020_5,ten_2020_7,ten_2020_10,ten_2020_11,ten_2020_12
team_id_impute,ten_2020,ten_2020,ten_2020,ten_2020,ten_2020
player_name,aaronbrewer,aaronbrewer,aaronbrewer,aaronbrewer,aaronbrewer
player_team,aaronbrewer_ten,aaronbrewer_ten,aaronbrewer_ten,aaronbrewer_ten,aaronbrewer_ten
player,aaronbrewer,aaronbrewer,aaronbrewer,aaronbrewer,aaronbrewer
position,g,g,g,g,g
team_name,ten,ten,ten,ten,ten
player_game_count,1,1,1,1,1


### OL Position Weights and Stats Groupby's

In [69]:
roll_oline.columns

Index(['unique_id', 'player_id', 'unique_team_id', 'team_id_impute',
       'player_name', 'player_team', 'player', 'position', 'team_name',
       'player_game_count', 'block_percent', 'declined_penalties',
       'franchise_id', 'grades_offense', 'grades_pass_block',
       'grades_run_block', 'hits_allowed', 'hurries_allowed',
       'non_spike_pass_block', 'non_spike_pass_block_percentage',
       'pass_block_percent', 'pbe', 'penalties', 'pressures_allowed',
       'sacks_allowed', 'snap_counts_block', 'snap_counts_ce',
       'snap_counts_lg', 'snap_counts_lt', 'snap_counts_offense',
       'snap_counts_pass_block', 'snap_counts_pass_play', 'snap_counts_rg',
       'snap_counts_rt', 'snap_counts_run_block', 'snap_counts_te', 'week',
       'year'],
      dtype='object')

In [70]:

ol_df.columns = [str(col) + '_ols' for col in ol_df.columns]
roll_oline = pd.merge(roll_oline, ol_df, left_on='player_id', right_on='unique_id_ols', how='left')

roll_oline.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)


def snap_fix(nData, var=None):
    if nData[var] == 0:
        return .1
    else:
        return nData[var]

roll_oline['snap_counts_offense'] = roll_oline.apply(lambda df: snap_fix(df, var='snap_counts_offense'), axis=1)
roll_oline['snap_counts_pass_block'] = roll_oline.apply(lambda df: snap_fix(df, var='snap_counts_pass_block'), axis=1) 
roll_oline['snap_counts_run_block'] = roll_oline.apply(lambda df: snap_fix(df, var='snap_counts_run_block'), axis=1) 


def weighted(nData, snap_Var='snap_counts_offense'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
ol_pff_off = roll_oline.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='snap_counts_pass_block'):
    pff_cols = nData.filter(like='grades_pass_block').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
ol_pff_pass = roll_oline.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='snap_counts_run_block'):
    pff_cols = nData.filter(like='grades_run_block').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
ol_pff_run = roll_oline.groupby('unique_team_id').apply(weighted).reset_index()

def pros(nData, snap_Var='snap_counts_offense'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)


roll_oline_avgs=roll_oline.fillna(roll_oline.mean())
roll_oline_avgs = roll_oline_avgs.groupby('unique_team_id').apply(pros).reset_index()

ol_sums = game_combine(data=roll_oline)

ol_stats = pd.merge(ol_pff_off, roll_oline_avgs, on='unique_team_id', how='left').merge(ol_sums, on='unique_team_id', how='left').merge(ol_pff_pass, on='unique_team_id', how='left').merge(ol_pff_run, on='unique_team_id', how='left')				
ol_stats.head()


,unique_team_id,grades_offense,grades_pass_block_x,grades_run_block_x,height_ols,wt_ols,speed_ols,draft_yr_ols,round_ols,selection_ols,...,snap_counts_lt,snap_counts_offense,snap_counts_pass_block,snap_counts_pass_play,snap_counts_rg,snap_counts_rt,snap_counts_run_block,snap_counts_te,grades_pass_block_y,grades_run_block_y
0,ari_2006_10,63.512075,68.271527,62.075212,76.276274,325.671975,5.269737,2001.595541,5.413217,151.570860,...,9.809524,59.809524,35.404762,35.452381,20.476190,9.809524,24.404762,0.095238,68.430603,62.074049
1,ari_2006_11,63.275788,67.601356,61.416593,76.064835,327.285714,5.266286,2002.769231,4.961538,136.130769,...,10.333333,50.555556,32.944444,33.000000,14.222222,10.333333,17.611111,0.166667,67.665601,61.440904
2,ari_2006_12,63.991576,67.794372,61.601561,76.000000,328.067538,5.280200,2003.248366,4.798475,130.846405,...,12.733333,61.200000,38.533333,38.533333,16.466667,12.733333,22.666667,0.000000,67.769089,61.577745
3,ari_2006_13,59.184875,66.462835,57.273862,76.004405,327.916300,5.278992,2003.211454,4.856828,132.746696,...,10.333333,50.444444,36.405556,36.388889,10.333333,10.333333,14.055556,0.222222,66.472240,57.305138
4,ari_2006_14,61.760000,67.420000,59.533333,76.000000,327.600000,5.277356,2003.200000,5.000000,137.400000,...,12.600000,63.000000,39.666667,39.666667,12.600000,12.600000,23.333333,0.000000,67.420000,59.533333


### Defensive Position Weights and Stats Groupby's

In [71]:
def_lineman.columns

Index(['unique_id', 'player_id', 'unique_team_id', 'team_id_impute',
       'player_name', 'player_team', 'player', 'position', 'team_name',
       'player_game_count', 'assists', 'batted_passes', 'catch_rate',
       'declined_penalties', 'forced_fumbles', 'franchise_id',
       'grades_coverage_defense', 'grades_defense', 'grades_defense_penalty',
       'grades_pass_rush_defense', 'grades_run_defense', 'grades_tackle',
       'hits', 'hurries', 'interceptions', 'longest', 'missed_tackle_rate',
       'missed_tackles', 'pass_break_ups', 'penalties', 'qb_rating_against',
       'receptions', 'sacks', 'snap_counts_box', 'snap_counts_corner',
       'snap_counts_coverage', 'snap_counts_defense', 'snap_counts_dl',
       'snap_counts_dl_a_gap', 'snap_counts_dl_b_gap',
       'snap_counts_dl_outside_t', 'snap_counts_dl_over_t', 'snap_counts_fs',
       'snap_counts_offball', 'snap_counts_pass_rush',
       'snap_counts_run_defense', 'snap_counts_slot', 'stops', 'tackles',
       'targets'

In [72]:
	   
def_lb.columns = [str(col) + '_def_lbs' for col in def_lb.columns]
def_linebackers = pd.merge(def_linebackers, def_lb, left_on='player_id', right_on='unique_id_def_lbs', how='left')
def_linebackers.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

def_dl.columns = [str(col) + '_def_dls' for col in def_dl.columns]
def_lineman = pd.merge(def_lineman, def_dl, left_on='player_id', right_on='unique_id_def_dls', how='left')
def_lineman.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

def_db.columns = [str(col) + '_def_dbs' for col in def_db.columns]
def_dbs = pd.merge(def_dbs, def_db, left_on='player_id', right_on='unique_id_def_dbs', how='left')
def_dbs.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)

	   
def snap_fixs(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]
roll_defense['snap_counts_defense'] = roll_defense.apply(lambda df: snap_fixs(df, var='snap_counts_defense'), axis=1) 
roll_defense['snap_counts_run_defense'] = roll_defense.apply(lambda df: snap_fixs(df, var='snap_counts_run_defense'), axis=1) 
roll_defense['snap_counts_pass_rush'] = roll_defense.apply(lambda df: snap_fixs(df, var='snap_counts_pass_rush'), axis=1) 
roll_defense['snap_counts_coverage'] = roll_defense.apply(lambda df: snap_fixs(df, var='snap_counts_coverage'), axis=1) 

def_linebackers['snap_counts_defense'] = def_linebackers.apply(lambda df: snap_fixs(df, var='snap_counts_defense'), axis=1) 
def_lineman['snap_counts_defense'] = def_lineman.apply(lambda df: snap_fixs(df, var='snap_counts_defense'), axis=1) 
def_dbs['snap_counts_defense'] = def_dbs.apply(lambda df: snap_fixs(df, var='snap_counts_defense'), axis=1) 


def weighted(nData, snap_Var='snap_counts_defense'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
def_pff_avg = roll_defense.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='snap_counts_run_defense'):
    pff_cols = nData.filter(like='grades_run_defense').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
def_pff_rush = roll_defense.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='snap_counts_pass_rush'):
    pff_cols = nData.filter(like='grades_pass_rush_defense').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
def_pff_passrush = roll_defense.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='snap_counts_coverage'):
    pff_cols = nData.filter(like='grades_coverage_defense').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)
def_pff_cov = roll_defense.groupby('unique_team_id').apply(weighted).reset_index()

def_sums = game_combine(data=roll_defense)

def_stats = pd.merge(def_pff_avg, def_pff_rush, on='unique_team_id', how='inner').merge(def_pff_passrush, on='unique_team_id', how='inner').merge(def_pff_cov, on='unique_team_id', how='inner').merge(def_sums, on='unique_team_id', how='inner')

def posgroup_weighted(nData, snap_Var='snap_counts_defense'):
    pff_cols = nData.filter(like='grades_').columns
    return pd.Series(np.average(nData[pff_cols], weights=nData[snap_Var], axis=0), pff_cols)


def pros(nData, snap_Var='snap_counts_defense'):
    pro_cols = nData.filter(regex='height|wt|speed|draft_yr|round|selection').columns
    return pd.Series(np.average(nData[pro_cols], weights=nData[snap_Var], axis=0), pro_cols)


def_linebackers_avgs=def_linebackers.fillna(def_linebackers.mean())
def_linebackers_avgs = def_linebackers_avgs.groupby('unique_team_id').apply(pros).reset_index()

def_lineman_avgs=def_lineman.fillna(def_lineman.mean())
def_lineman_avgs = def_lineman_avgs.groupby('unique_team_id').apply(pros).reset_index()

def_dbs_avgs=def_dbs.fillna(def_dbs.mean())
def_dbs_avgs = def_dbs_avgs.groupby('unique_team_id').apply(pros).reset_index()


lbs_pff_avg = def_linebackers.groupby('unique_team_id').apply(posgroup_weighted)
Fant_colmean = str('_'+'lbs_pff_avgs')
lbs_pff_avg.rename(columns=lambda x: x+Fant_colmean, inplace=True)
lbs_pff_avg = lbs_pff_avg.reset_index()

dl_pff_avg = def_lineman.groupby('unique_team_id').apply(posgroup_weighted)				
Fant_colmean = str('_'+'dls_pff_avgs')
dl_pff_avg.rename(columns=lambda x: x+Fant_colmean, inplace=True)
dl_pff_avg = dl_pff_avg.reset_index()

dbs_pff_avg = def_dbs.groupby('unique_team_id').apply(posgroup_weighted)				
Fant_colmean = str('_'+'dbs_pff_avgs')
dbs_pff_avg.rename(columns=lambda x: x+Fant_colmean, inplace=True)
dbs_pff_avg = dbs_pff_avg.reset_index()

    

/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [73]:
spread_comb.columns

Index(['home_matchup_id', 'away_matchup_id', 'team_id', 'schedule_date',
       'schedule_season', 'schedule_week', 'team_home_abb', 'score_home',
       'score_away', 'away_team_abb', 'team_favorite_id', 'spread_favorite',
       'over_under_line', 'fav_spread', 'team_notfav_id', 'fav_cover',
       'over_under_result', 'precip', 'dome', 'temperature', 'wind_mph',
       'team_abb', 'fav_homeoraway', 'team_ids', 'team_conference',
       'team_division', 'team', 'w-l', 'year', 'week', 'off_dvoa_rank',
       'off_dvoa', 'off_weight_dvoa_rank', 'off_weight_dvoa', 'pass_off_rank',
       'pass_off', 'rush_off_rank', 'rush_off', 'def_dvoa_rank', 'def_dvoa',
       'def_weight_dvoa_rank', 'def_weight_dvoa', 'pass_def_rank', 'pass_def',
       'rush_def_rank', 'rush_def', 'off_dvoa_momentum',
       'off_wt_dvoa_momentum', 'pass_off_dvoa_momentum',
       'rush_off_dvoa_momentum', 'def_dvoa_momentum', 'def_wt_dvoa_momentum',
       'pass_def_dvoa_momentum', 'rush_dvoa_momentum'],
      dty

In [118]:
spread_comb.tail().T

,8461,8462,8463,8464,8465
home_matchup_id,lvvscin_2021_11,kcvsdal_2021_11,seavsari_2021_11,lacvspit_2021_11,tbvsnyg_2021_11
away_matchup_id,cin@lv_2021_11,dal@kc_2021_11,ari@sea_2021_11,pit@lac_2021_11,nyg@tb_2021_11
team_id,cin_2021_11,dal_2021_11,ari_2021_11,pit_2021_11,nyg_2021_11
schedule_date,nan,nan,nan,nan,nan
schedule_season,2021,2021,2021,2021,2021
schedule_week,11,11,11,11,11
team_home_abb,lv,kc,sea,lac,tb
score_home,nan,nan,nan,nan,nan
score_away,nan,nan,nan,nan,nan
away_team_abb,cin,dal,ari,pit,nyg


### Bring all of these together into one line to represent a team_year_week momentum indicator

In [120]:

from functools import reduce

spread_vars = spread_comb[spread_comb['schedule_week'] != '1']
spread_ids = spread_vars[['team_id', 'home_matchup_id']]
spread_ids.columns = ['unique_team_id','home_matchup_id']
spread_targs = spread_vars[['team_id','schedule_week','schedule_season','home_matchup_id','team_favorite_id','spread_favorite','over_under_line','fav_cover','over_under_result']]
spread_divs = spread_vars[['team_id','fav_homeoraway','team_conference','team_division',
                          "temperature",
                          "wind_mph",
                          "dome",
                          "precip"]]
spread_dvoa= spread_vars[['team_id','off_dvoa_rank',
       'off_dvoa', 'off_weight_dvoa_rank', 'off_weight_dvoa', 'pass_off_rank',
       'pass_off', 'rush_off_rank', 'rush_off', 'def_dvoa_rank', 'def_dvoa',
       'def_weight_dvoa_rank', 'def_weight_dvoa', 'pass_def_rank', 'pass_def',
       'rush_def_rank', 'rush_def','off_dvoa_momentum',
       'off_wt_dvoa_momentum', 'pass_off_dvoa_momentum',
       'rush_off_dvoa_momentum', 'def_dvoa_momentum', 'def_wt_dvoa_momentum',
       'pass_def_dvoa_momentum', 'rush_dvoa_momentum']]
spread_dvoa.rename(columns={'team_id':'unique_team_id'}, inplace=True)
spread_divs.rename(columns={'team_id':'unique_team_id'}, inplace=True)
#'fav_conf_game','fav_div_game',
dfs_list = [spread_ids,
            spread_divs,
            spread_dvoa,
            tgs_roll,
            qb_stats,
            qb_rush_stats,
            wr_stats,
            rb_stats,
            rb_rec_stats,
            te_stats,
            ol_stats,
            dbs_pff_avg,
            dl_pff_avg,
            lbs_pff_avg,
            def_linebackers_avgs,
            def_lineman_avgs,
            def_dbs_avgs,
            def_stats]

dfs_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'],
                                            how='left'), dfs_list)

def fav_ids(nData):
    if str(nData['team_favorite_id']) in str(nData['team_id']):
        return nData['team_id']
    else:
        pass
spread_targs['fav_team_id'] = spread_targs.apply(lambda nData: fav_ids(nData), axis=1)


favs = spread_targs[~spread_targs['fav_team_id'].isnull()]
not_fav = spread_targs[spread_targs['fav_team_id'].isnull()]

not_fav_df = dfs_team[dfs_team.unique_team_id.isin(not_fav.team_id)]

favs = favs[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]
not_fav = not_fav[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]


fin_df = pd.merge(favs, dfs_team, left_on='team_id', right_on='unique_team_id', how='left').merge(not_fav_df, on='home_matchup_id', how='left')#.merge(spread_divs, on='team_id', how='left')

/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [75]:
from functools import reduce

spread_targs = spread_vars[['team_id','schedule_week','schedule_season','team_favorite_id','spread_favorite','over_under_line','over_under_result']]
spread_targs.rename(columns={'team_id':'unique_team_id'}, inplace=True)
spread_divs = spread_vars[['team_id','fav_homeoraway',
                          "temperature",
                          "wind_mph",
                          "dome",
                          "precip"]]


defs_pff_list = [spread_ids,
            tgs_roll,
            qb_stats,
            qb_rush_stats,
            wr_stats,
            rb_stats,
            rb_rec_stats,
            te_stats,
            ol_stats,
            dbs_pff_avg,
            dl_pff_avg,
            lbs_pff_avg,
            def_linebackers_avgs,
            def_lineman_avgs,
            def_dbs_avgs,
            def_stats]

defs_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'],
                                            how='left'), defs_pff_list)

def fav_ids(nData):
    if str(nData['team_favorite_id']) in str(nData['unique_team_id']):
        return nData['unique_team_id']
    else:
        pass
spread_targs['fav_team_id'] = spread_targs.apply(lambda nData: fav_ids(nData), axis=1)

favs = spread_targs[~spread_targs['fav_team_id'].isnull()]
favs['over_under_line'] = (favs['over_under_line'].apply(float))
favs['spread_favorite'] = (favs['spread_favorite'].apply(float))
favs['implied_score'] = (favs['over_under_line']/2)-favs['spread_favorite']

not_fav = spread_targs[spread_targs['fav_team_id'].isnull()]
not_fav['over_under_line'] = (not_fav['over_under_line'].apply(float))
not_fav['spread_favorite'] = (not_fav['spread_favorite'].apply(float))
not_fav['spread_favorite'] = (not_fav['spread_favorite']*-1)
not_fav['implied_score'] = (not_fav['over_under_line']/2)-not_fav['spread_favorite']


favs = favs[['unique_team_id','schedule_week','schedule_season','spread_favorite','over_under_line','over_under_result','implied_score']]
not_fav = not_fav[['unique_team_id','schedule_week','schedule_season','spread_favorite','over_under_line','over_under_result','implied_score']]

comb = pd.concat([favs, not_fav], axis=0)

OU_df = pd.merge(comb, defs_team, on='unique_team_id', how='left').merge(defs_team, on='opponent_id_overall_pff', how='left')



/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tom/anaconda3/envs/cloak/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [76]:

not_fav['implied_score2'] = (not_fav['over_under_line']/2)-not_fav['spread_favorite']
not_fav.head(n=5)

,unique_team_id,schedule_week,schedule_season,spread_favorite,over_under_line,over_under_result,implied_score,implied_score2
19,sf_2006_2,2,2006,3.0,44.5,under,19.25,19.25
22,gb_2006_2,2,2006,1.0,40.5,over,19.25,19.25
25,nyj_2006_2,2,2006,6.0,38.5,over,13.25,13.25
30,jax_2006_2,2,2006,2.5,36.5,under,15.75,15.75
35,no_2006_3,3,2006,4.0,45.0,under,18.50,18.50


In [77]:
OU_df.tail(n=5).T

,10995,10996,10997,10998,10999
unique_team_id_x,dal_2021_11,ari_2021_11,pit_2021_11,nyg_2021_11,nyg_2021_11
schedule_week,11,11,11,11,11
schedule_season,2021,2021,2021,2021,2021
spread_favorite,2.5,3,6,10.5,10.5
over_under_line,56.5,47.5,47,49.5,49.5
...,...,...,...,...,...
total_pressures_y,0.650794,0.746032,0.647059,0.85,1.11404
touchdowns_y,0.031746,0.0793651,0.0196078,0.0833333,0.0350877
yards_y,8.80952,10.8095,10.5686,11.7917,10.886
yards_after_catch_y_y,4.47619,6.1746,4.2451,6.325,8.15789


In [78]:
OU_df['overall_perf_by_overall_perf'] = OU_df['overall_performance_overall_pff_x']/OU_df['overall_performance_overall_pff_y']
OU_df['overall_off_vs_overall_def'] = OU_df['offense_overall_pff_x']/OU_df['defense_overall_pff_y']
OU_df['overall_pass_vs_overall_coverage'] = OU_df['pass_overall_pff_x']/OU_df['coverage_overall_pff_y']
OU_df['overall_passblock_vs_overall_passrush'] = OU_df['pass_blocking_overall_pff_x']/OU_df['pass_rush_overall_pff_y']
OU_df['overall_rec_vs_overall_coverage'] = OU_df['receiving_overall_pff_x']/OU_df['coverage_overall_pff_y']
OU_df['overall_rush_vs_overall_rushdef'] = OU_df['rushing_overall_pff_x']/OU_df['rush_defense_overall_pff_y']
OU_df['overall_runblocking_vs_overall_rushdef'] = OU_df['run_blocking_overall_pff_x']/OU_df['rush_defense_overall_pff_y']
OU_df['overall_rush_vs_overall_tackling'] = OU_df['rushing_overall_pff_x']/OU_df['tackling_overall_pff_y']
OU_df['overall_tackling_vs_overall_tackling'] = OU_df['tackling_overall_pff_x']/OU_df['tackling_overall_pff_y']
OU_df['overall_off_vs_overall_off'] = OU_df['offense_overall_pff_x']/OU_df['offense_overall_pff_y']
OU_df['overall_def_vs_overall_def'] = OU_df['defense_overall_pff_x']/OU_df['defense_overall_pff_y']
OU_df['overall_passrush_vs_overall_passrush'] = OU_df['pass_rush_overall_pff_x']/OU_df['pass_rush_overall_pff_y']

In [79]:
OU_df = OU_df.drop_duplicates(subset=['unique_team_id_x'], keep='first')
OU_df=OU_df.round(2)
OU_df.tail(n=2).T
OU_df.to_csv(cur_dir+'/modeling_datasets/fin_df_week'+cur_week+'_OUv1.csv', index=False)

spread_vars = spread_comb[spread_comb['schedule_week'] != '1']
spread_ids = spread_vars[['team_id', 'home_matchup_id']]
spread_ids.columns = ['unique_team_id','home_matchup_id']
spread_targs = spread_vars[['team_id','schedule_week','schedule_season','home_matchup_id','team_favorite_id','spread_favorite','over_under_line','fav_cover','over_under_result','team_home_abb','away_team_abb','score_home','score_away']]

dfs_list = [spread_ids,
            qb_stats,
            wr_stats,
            rb_stats,
            ol_stats,
            dbs_pff_avg,
            dl_pff_avg,
            lbs_pff_avg,
            def_stats]

defs_list = [spread_ids,
            dbs_pff_avg,
            dl_pff_avg,
            lbs_pff_avg,
            def_stats]

dfs_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'],
                                            how='left'), dfs_list)
def_team = reduce(lambda  left,right: pd.merge(left,right,on=['unique_team_id'],
                                            how='left'), defs_list)

def team_points(nData):
    if str(nData['team_home_abb']) in str(nData['team_id']):
        return nData['score_home']
    elif str(nData['away_team_abb']) in str(nData['team_id']):
        return nData['score_away']
    else:
        pass
spread_targs['team_score'] = spread_targs.apply(lambda nData: team_points(nData), axis=1)

OU_df = pd.merge(spread_targs, dfs_team, left_on='team_id', right_on='unique_team_id', how='left')
OU_df.head()


In [80]:
#not_fav = not_fav[['team_id','schedule_week','schedule_season','spread_favorite','over_under_line','fav_cover','over_under_result']]
#ou_df1 = pd.merge(favs, dfs_team, left_on='team_id', right_on='unique_team_id', how='left')
#ou_df2 = pd.merge(not_fav, dfs_team, left_on='team_id', right_on='unique_team_id', how='left')

#conc =pd.concat([ou_df1, ou_df2], axis=0)
##conc = conc.drop_duplicates(keep='first')
#conc.to_csv(cur_dir+'/modeling_datasets/fin_df_ou_v1.csv', index=False)

In [121]:
colNames = fin_df.columns[fin_df.columns.str.contains(pat = 'pff_')]
pff_dfs = fin_df[colNames]

### Fav-Passing Ratio to Overall Defense and DB Performance

fin_df['overall_perf_by_overall_perf'] = fin_df['overall_performance_overall_pff_x']/fin_df['overall_performance_overall_pff_y']
fin_df['overall_off_vs_overall_def'] = fin_df['offense_overall_pff_x']/fin_df['defense_overall_pff_y']
fin_df['overall_pass_vs_overall_coverage'] = fin_df['pass_overall_pff_x']/fin_df['coverage_overall_pff_y']
fin_df['overall_passblock_vs_overall_passrush'] = fin_df['pass_blocking_overall_pff_x']/fin_df['pass_rush_overall_pff_y']
fin_df['overall_rec_vs_overall_coverage'] = fin_df['receiving_overall_pff_x']/fin_df['coverage_overall_pff_y']
fin_df['overall_rush_vs_overall_rushdef'] = fin_df['rushing_overall_pff_x']/fin_df['rush_defense_overall_pff_y']
fin_df['overall_runblocking_vs_overall_rushdef'] = fin_df['run_blocking_overall_pff_x']/fin_df['rush_defense_overall_pff_y']
fin_df['overall_rush_vs_overall_tackling'] = fin_df['rushing_overall_pff_x']/fin_df['tackling_overall_pff_y']
fin_df['overall_tackling_vs_overall_tackling'] = fin_df['tackling_overall_pff_x']/fin_df['tackling_overall_pff_y']
fin_df['overall_off_vs_overall_off'] = fin_df['offense_overall_pff_x']/fin_df['offense_overall_pff_y']
fin_df['overall_def_vs_overall_def'] = fin_df['defense_overall_pff_x']/fin_df['defense_overall_pff_y']
fin_df['overall_passrush_vs_overall_passrush'] = fin_df['pass_rush_overall_pff_x']/fin_df['pass_rush_overall_pff_y']


fin_df['fav_rush_ovr_ratio_opposing_def'] = fin_df['rush_pff_off_x']/fin_df['def_pff_def_y']
fin_df['fav_rush_run_ratio_opposing_def'] = fin_df['rush_pff_run_x']/fin_df['def_pff_rushdef_y']
fin_df['fav_rush_ovr_ratio_opposing_dls'] = fin_df['pass_pff_off_x']/fin_df['def_pff_def_dls_pff_avgs_y']
fin_df['fav_rush_run_ratio_opposing_dls'] = fin_df['pass_pff_pass_x']/fin_df['def_pff_rushdef_dls_pff_avgs_y']
fin_df['fav_rush_ovr_ratio_opposing_lbs'] = fin_df['pass_pff_off_x']/fin_df['def_pff_def_lbs_pff_avgs_y']
fin_df['fav_rush_run_ratio_opposing_lbs'] = fin_df['pass_pff_pass_x']/fin_df['def_pff_rushdef_lbs_pff_avgs_y']

In [122]:
fin_df=fin_df.round(2)
fin_df.tail(n=2).T

,8951,8952
team_id,mia_2021_11,cin_2021_11
schedule_week,11,11
schedule_season,2021,2021
spread_favorite,-3.5,-1.5
over_under_line,44.5,50.0
...,...,...
total_pressures_y,0.48,0.89
touchdowns_y,0.11,0.11
yards_y,11.94,10.78
yards_after_catch_y_y,5.73,5.33


In [83]:
fin_df = fin_df.drop(['unique_team_id_x','team_division','team_conference','unique_team_id_y','opponent_id_x','opponent_id_y','opponent_id_overall_pff_x','opponent_id_overall_pff_y','wl_y'], axis=1)

In [84]:
fin_df = fin_df.drop_duplicates(subset=['team_id'], keep='first')
fin_df.to_csv(cur_dir+'/modeling_datasets/fin_df_week'+cur_week+'_spreads_new_ats_fo_newmeanv2.csv', index=False)

In [85]:











spread_comb.to_csv(cur_dir+'/spead_comb.csv', index=False)
